In [1]:
import warnings
warnings.filterwarnings('ignore')

254.66290897019599

In [1]:
import numpy as np
import pandas as pd
import math

class Pension:
    
    def __init__(self, wage_history=None, delivery_year=None, military_year=None, rp_entitlement_year=None):
        
        self.wage_history = wage_history
        self.delivery_year = delivery_year
        self.military_year = military_year
        self.p = {
            1988: 2.4, 1989: 2.4, 1990: 2.4, 1991: 2.4, 1992: 2.4, 1993: 2.4, 1994: 2.4, 1995: 2.4, 1996: 2.4, 1997: 2.4, 1998: 2.4,
            1999: 1.8, 2000: 1.8, 2001: 1.8, 2002: 1.8, 2003: 1.8, 2004: 1.8, 2005: 1.8, 2006: 1.8, 2007: 1.8,
            2008: 1.5, 2009: 1.485, 2010: 1.470, 2011: 1.455, 2012: 1.440, 2013: 1.425, 2014: 1.410, 2015: 1.395, 2016: 1.380, 2017: 1.365, 2018: 1.350, 2019: 1.335, 2020: 1.320,
            2021: 1.305, 2022: 1.290, 2023: 1.275, 2024: 1.260, 2025: 1.245, 2026: 1.230, 2027: 1.215, 2028: 1.2
        }
        
        # 2020 기준으로 A(실질임금 상승률을 반영한 2018-2020 평균 가입자평균소득월액) 계산
        self.A = sum([259.9734, 259.9734/1.021, 259.9734/1.021**2]) / 3 
        self.delivery_credit = 1
        self.military_credit = 0.5
        self.rp_enforcement_year = 2020
        self.rp_entitlement_year = rp_entitlement_year
    
    def national_pension(self):
        
        """
        물가상승률 2%
        실질임금상승률 2.1%
        실질금리 2%
        """
        
        # 수급시점 기준으로 연금액을 산정: 수급연령 65세
        entitlement_year = self.wage_history.iloc[-1, 0] + 1
        print(f"수급시점: {entitlement_year}")
        print(f"2020년 기준 A값: {self.A}")
        
        # # A: 연금수급 전 3년간의 평균소득월액의 평균액(실질임금상승률 2.1% 반영)
        # # A는 2020년 기준 수급시점의 직전 3년간 평균소득월액의 평균액
        # A = self.A * 1.021 ** (entitlement_year - 2020)
        # print(f"실질임금상승률을 반영한 수급시점의 A값: {A}")
        A = self.A
        
        # 월소득 -> 기준소득월액
        self.wage_history['wage_with_upper_limit'] = self.wage_history['wage']\
                                                        .apply(lambda wage: min(wage, 504))
        
            
        # 2020년 기준으로 B 계산 후 수급시점 기준으로 환산
        self.wage_history['standard_wage'] = self.wage_history['wage_with_upper_limit']\
                                                .apply(lambda wage: math.floor(wage * 10) / 10 if not pd.isna(wage) else wage)
        
        B = self.wage_history['standard_wage'].mean()
        # B_base: 2020년 기준 가입자 개인의 가입기간 중 기준소득월액의 평균액
        # B_base = self.wage_history['standard_wage'].mean()
        
        # # B: 실질임금상승률을 반영한 수급시점의 B값
        # B = B_base * 1.021 ** (entitlement_year - 2020)
        # print(f"2020년 기준 B값: {B_base}")
        # print(f"B: {B}")
        print(f"2020년 기준 B값: {B}")
        
        # C, Y 계산
        if self.delivery_year:
            if self.delivery_year > 2028:
                Y = 1.2
            else:
                Y = self.p[self.delivery_year]
            C = self.delivery_credit
        elif self.military_year:
            Y = self.p[self.military_year]
            C = self.military_credit
        else:
            Y = 0
            C = 0
        print(f"Y: {Y}\nC: {C}")
        
        # X 계산
        X = 1.2
        print(f"X: {X}")
        
        # n 계산: 실업기간에는 국민연금 미가입 상태로 가정
        P = self.wage_history['standard_wage'].count()
        n = max(0, P - 20)
        print(f"P: {P}\nn: {n}")
        
        # P_n 계산
        self.wage_history['P_n'] = self.wage_history\
                                    .apply(lambda row: self.p[row['year']] if row['year'] <= 2028 else 1.2, axis=1)
        
        # A+B 계산
        self.wage_history['A+B'] = self.wage_history\
                                    .apply(lambda row: A+B if row['year'] >= 1999 else A+0.75*B, axis=1)
        
        # A+B x P_n
        self.wage_history['(A+B)*P_n'] = self.wage_history\
                                    .apply(lambda row: row['A+B'] * row['P_n'] if not pd.isna(row['standard_wage']) else np.nan, axis=1)
        
        intermediate_sum = self.wage_history['(A+B)*P_n'].sum()
        total = (intermediate_sum + (Y * (A + A) * C) + (X * (A + (1/2 * A))* 6)) * (1 + 0.05*n) / P
        print(f"2020년 기준 수급시점({entitlement_year})의 노령연금액: {total}")
        
        # deflated_total = total / 1.021**(entitlement_year-2020)
        # print(f"2020년 가치로 환산한 수급시점 {entitlement_year}의 노령연금액(annuity): {deflated_total}")
        r = 1 + 0.02 / 12
        monthly = (1-r) / (1 - r**12) * total
        # monthly = (1-r) / (1 - r**12) * deflated_total
        print(f"2020년 가치로 환산한 수급시점 {entitlement_year}의 노령연금 월지급액(annuity): {monthly}")
        
        return B, pd.DataFrame.from_dict({'국민연금 월지급액': [monthly], '생애평균소득': [B], '소득대체율': [monthly / B], '평균임금대비': [monthly / A]})
    
    def dc(self):
        """
        dc 방식
        적립률 8.3% + 정부 4.17% 보조
        연금비율 5%
        실질수익률 4%
        보험요율 8.3%
        실질금리 2%
        """
        self.wage_history['annual_saving'] = self.wage_history\
                                                .apply(lambda row: 12 * row['wage'] * 0.0417 * 3 if row['year'] >= self.rp_enforcement_year else np.nan, axis=1)
        
        self.wage_history['annual_reserve'] = self.wage_history\
                                                .apply(lambda row: row['annual_saving'] * 1.04 ** (self.rp_entitlement_year - row['year']), axis=1)
        # print(self.wage_history.head())
        rp_fund = self.wage_history['annual_reserve'].sum()
        print(f"퇴직연금 적립액: {rp_fund}")
        
        rp_annuity = 0.05 * rp_fund
        r = 1 + 0.02 / 12
        rp_monthly = (1-r) / (1 - r**12) * rp_annuity
        print(f"퇴직연금 월지급액: {rp_monthly}")
        np_result = self.national_pension()
        A = self.A
        B = np_result[0]
        np_monthly = np_result[1].iloc[0, 0]
        
        return pd.concat([np_result[1], pd.DataFrame.from_dict({'퇴직연금 월지급액': [rp_monthly], '합산연금액': [rp_monthly + np_monthly], '합산소득대체율': [(rp_monthly+np_monthly) / B], '평균임금대비': [(rp_monthly+np_monthly) / A]})], axis=1)

In [20]:
df_2000 = pd.read_csv('../data/new/wage_history_2000.csv')
df_2000.tail()

,Unnamed: 0,year,age,group_1,group_2,group_3,group_4,group_5,group_6
41,41,2060,60,NaN,NaN,NaN,191.899107,241.49699,193.389172
42,42,2061,61,NaN,NaN,NaN,188.067511,NaN,NaN
43,43,2062,62,NaN,NaN,NaN,184.129481,NaN,NaN
44,44,2063,63,NaN,NaN,NaN,180.297885,NaN,NaN
45,45,2064,64,NaN,NaN,NaN,176.466289,NaN,NaN


In [21]:
df_2000_group_1 = df_2000[['year', 'group_1']]
df_2000_group_1.columns = ['year', 'wage']
df_2000_group_2 = df_2000[['year', 'group_2']]
df_2000_group_2.columns = ['year', 'wage']
df_2000_group_3 = df_2000[['year', 'group_3']]
df_2000_group_3.columns = ['year', 'wage']
df_2000_group_4 = df_2000[['year', 'group_4']]
df_2000_group_4.columns = ['year', 'wage']
df_2000_group_5 = df_2000[['year', 'group_5']]
df_2000_group_5.columns = ['year', 'wage']
df_2000_group_6 = df_2000[['year', 'group_6']]
df_2000_group_6.columns = ['year', 'wage']

In [23]:
pension_2000_group_1 = Pension(df_2000_group_1, rp_entitlement_year=2055)
pension_2000_group_2 = Pension(df_2000_group_2, military_year=2021, rp_entitlement_year=2056)
pension_2000_group_3 = Pension(df_2000_group_3, rp_entitlement_year=2055)
pension_2000_group_4 = Pension(df_2000_group_4, delivery_year=2029, rp_entitlement_year=2065)
pension_2000_group_5 = Pension(df_2000_group_5, military_year=2020, rp_entitlement_year=2061)
pension_2000_group_6 = Pension(df_2000_group_6, delivery_year=2029, rp_entitlement_year=2061)

In [24]:
pension_2000_group_1.national_pension()

수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2065)의 노령연금액: 1173.534535057538
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 96.90132906026375


/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['wage_with_upper_limit'] = self.wage_history['wage']\
/tmp/ipykernel_13182/4094718912.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['standard_wage'] = self.wage_history['wage_with_upper_limit']\
/tmp/ipykernel_13182/4094718912.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

(264.44848484848484,
    국민연금 월지급액     소득대체율    평균임금대비
 0  96.901329  0.366428  0.380508)

In [25]:
pension_2000_group_1.dc()

퇴직연금 적립액: 25142.20621295351
퇴직연금 월지급액: 103.80236476905063
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2065)의 노령연금액: 1173.534535057538
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 96.90132906026375


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\


,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,96.901329,0.366428,0.380508,103.802365,200.703694,264.448485,0.758952,0.788115


In [32]:
pension_2000_group_2.dc()

퇴직연금 적립액: 34464.2164428651
퇴직연금 월지급액: 142.28930971215445
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 410.8633333333334
Y: 1.305
C: 0.5
X: 1.2
P: 30
n: 10
2020년 기준 수급시점(2065)의 노령연금액: 1353.579395845747
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 111.76802942540657


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,111.768029,0.272032,0.438886,142.28931,254.057339,410.863333,0.61835,0.997622


In [33]:
pension_2000_group_3.dc()

퇴직연금 적립액: 19358.290723224374
퇴직연금 월지급액: 79.92283326043933
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 226.05806451612904
Y: 0
C: 0
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2065)의 노령연금액: 1035.2643888296177
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 85.4840588065293


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,85.484059,0.378151,0.335675,79.922833,165.406892,226.058065,0.731701,0.649513


In [34]:
pension_2000_group_4.dc()

퇴직연금 적립액: 21150.93503508549
퇴직연금 월지급액: 87.32396254817465
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 172.9838709677419
Y: 1.2
C: 1
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2065)의 노령연금액: 976.9714041729387
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 80.67067878286873


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,80.670679,0.466348,0.316774,87.323963,167.994641,172.983871,0.971158,0.659675


In [35]:
pension_2000_group_5.dc()

퇴직연금 적립액: 25237.359892134788
퇴직연금 월지급액: 104.19521720339282
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 235.6516129032258
Y: 1.32
C: 0.5
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2065)의 노령연금액: 1074.0331872400095
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 88.68528380656922


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,88.685284,0.376341,0.348246,104.195217,192.880501,235.651613,0.818499,0.757395


In [36]:
pension_2000_group_6.dc()

퇴직연금 적립액: 14440.147551938126
퇴직연금 월지급액: 59.617738030202595
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 164.74285714285716
Y: 1.2
C: 1
X: 1.2
P: 21
n: 1
2020년 기준 수급시점(2065)의 노령연금액: 709.7400668553374
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 58.60479918661331


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,58.604799,0.355735,0.230127,59.617738,118.222537,164.742857,0.717619,0.464231


In [36]:
182.570443 * 0.0417 * 3 * 12

274.07474903160005

In [26]:
274.074748 * 1.06 ** (2055 - 2022)

1874.8329482993372

In [357]:
pension_2000_group_2.national_pension()

entitlement_year: 2065
A_base: 254.6629
A: 1197.516265523042
B_2020: 410.8633333333334
B: 1001.6249049707955
Y: 1.305
C: 0.5
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2065의 노령연금액(annuity): 1679.931611123641
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 138.7156500172784


,연금액,소득대체율,평균임금대비
0,138.71565,0.33762,0.436213


In [358]:
pension_2000_group_3.national_pension()

entitlement_year: 2065
A_base: 254.6629
A: 1197.516265523042
B_2020: 226.05806451612904
B: 551.0966032229281
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2065의 노령연금액(annuity): 1361.6166041075116
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 112.43167939840113


,연금액,소득대체율,평균임금대비
0,112.431679,0.497358,0.353559


In [359]:
pension_2000_group_4.national_pension()

entitlement_year: 2065
A_base: 254.6629
A: 1197.516265523042
B_2020: 172.9838709677419
B: 421.70945728801496
Y: 1.2
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2065의 노령연금액(annuity): 1337.387457961375
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 110.43102548203309


,연금액,소득대체율,평균임금대비
0,110.431025,0.638389,0.347267


In [360]:
pension_2000_group_5.national_pension()

entitlement_year: 2065
A_base: 254.6629
A: 1197.516265523042
B_2020: 235.6516129032258
B: 574.484275502174
Y: 1.32
C: 0.5
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2065의 노령연금액(annuity): 1417.9495692499027
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 117.08321629751383


,연금액,소득대체율,평균임금대비
0,117.083216,0.496849,0.368186


In [361]:
pension_2000_group_6.national_pension()

entitlement_year: 2065
A_base: 254.6629
A: 1197.516265523042
B_2020: 164.74285714285716
B: 401.6190670790732
Y: 1.2
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2065의 노령연금액(annuity): 928.2234601831808
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 76.64545377205877


,연금액,소득대체율,평균임금대비
0,76.645454,0.465243,0.241023


In [15]:
df_1990 = pd.read_csv('../data/new/wage_history_1990.csv')
df_1990.head()

,Unnamed: 0,year,age,group_1,group_2,group_3,group_4,group_5,group_6
0,0,2009,19,NaN,NaN,NaN,128.784204,NaN,128.784204
1,1,2010,20,NaN,NaN,NaN,133.467266,NaN,133.467266
2,2,2011,21,NaN,NaN,NaN,138.043894,NaN,138.043894
3,3,2012,22,182.570443,NaN,NaN,142.726956,176.253422,142.726956
4,4,2013,23,184.200432,NaN,NaN,147.303585,182.639416,147.303585


In [16]:
df_1990_group_1 = df_1990[['year', 'group_1']]
df_1990_group_1.columns = ['year', 'wage']
df_1990_group_2 = df_1990[['year', 'group_2']]
df_1990_group_2.columns = ['year', 'wage']
df_1990_group_3 = df_1990[['year', 'group_3']]
df_1990_group_3.columns = ['year', 'wage']
df_1990_group_4 = df_1990[['year', 'group_4']]
df_1990_group_4.columns = ['year', 'wage']
df_1990_group_5 = df_1990[['year', 'group_5']]
df_1990_group_5.columns = ['year', 'wage']
df_1990_group_6 = df_1990[['year', 'group_6']]
df_1990_group_6.columns = ['year', 'wage']

In [17]:
pension_1990_group_1 = Pension(df_1990_group_1, rp_entitlement_year=2045)
pension_1990_group_2 = Pension(df_1990_group_2, military_year=2011, rp_entitlement_year=2046)
pension_1990_group_3 = Pension(df_1990_group_3, rp_entitlement_year=2045)
pension_1990_group_4 = Pension(df_1990_group_4, delivery_year=2019, rp_entitlement_year=2055)
pension_1990_group_5 = Pension(df_1990_group_5, military_year=2010, rp_entitlement_year=2051)
pension_1990_group_6 = Pension(df_1990_group_6, delivery_year=2019, rp_entitlement_year=2051)

In [18]:
pension_1990_group_1.national_pension()

수급시점: 2055
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2055)의 노령연금액: 1218.3078927743993
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 100.59836373597676


/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['wage_with_upper_limit'] = self.wage_history['wage']\
/tmp/ipykernel_13182/4094718912.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['standard_wage'] = self.wage_history['wage_with_upper_limit']\
/tmp/ipykernel_13182/4094718912.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

(264.44848484848484,
     국민연금 월지급액     소득대체율    평균임금대비
 0  100.598364  0.380408  0.395026)

In [19]:
pension_1990_group_1.dc()

퇴직연금 적립액: 17778.479575734866
퇴직연금 월지급액: 73.40040911003084
수급시점: 2055
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2055)의 노령연금액: 1218.3078927743993
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 100.59836373597676


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\


,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,100.598364,0.380408,0.395026,73.400409,173.998773,264.448485,0.657968,0.683251


In [366]:
pension_1990_group_2.national_pension()

entitlement_year: 2055
A_base: 254.6629
A: 848.9418103527513
B_2020: 410.8633333333334
B: 821.6812879054058
Y: 1.455
C: 0.5
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2055의 노령연금액(annuity): 1602.4964588745636
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 132.32165909092566


,연금액,소득대체율,평균임금대비
0,132.321659,0.322058,0.416106


In [367]:
pension_1990_group_3.national_pension()

entitlement_year: 2055
A_base: 254.6629
A: 848.9418103527513
B_2020: 226.05806451612904
B: 452.091161520902
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2055의 노령연금액(annuity): 1280.3607340600456
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 105.72220339549723


,연금액,소득대체율,평균임금대비
0,105.722203,0.467677,0.33246


In [368]:
pension_1990_group_4.national_pension()

entitlement_year: 2055
A_base: 254.6629
A: 848.9418103527513
B_2020: 172.9838709677419
B: 345.9486363274974
Y: 1.335
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2055의 노령연금액(annuity): 1249.3688312503332
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 103.1631337791775


,연금액,소득대체율,평균임금대비
0,103.163134,0.596374,0.324412


In [369]:
pension_1990_group_5.national_pension()

entitlement_year: 2055
A_base: 254.6629
A: 848.9418103527513
B_2020: 235.6516129032258
B: 471.27719871321864
Y: 1.47
C: 0.5
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2055의 노령연금액(annuity): 1338.0823120455104
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 110.48840111286817


,연금액,소득대체율,평균임금대비
0,110.488401,0.468863,0.347448


In [370]:
pension_1990_group_6.national_pension()

entitlement_year: 2055
A_base: 254.6629
A: 848.9418103527513
B_2020: 164.74285714285716
B: 329.4675188757636
Y: 1.335
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2055의 노령연금액(annuity): 879.6390971721248
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 72.6337360026394


,연금액,소득대체율,평균임금대비
0,72.633736,0.440892,0.228408


In [7]:
df_1980 = pd.read_csv('../data/new/wage_history_1980.csv')
df_1980.head()

,Unnamed: 0,year,age,group_1,group_2,group_3,group_4,group_5,group_6
0,0,1999,19,NaN,NaN,NaN,128.784204,NaN,128.784204
1,1,2000,20,NaN,NaN,NaN,133.467266,NaN,133.467266
2,2,2001,21,NaN,NaN,NaN,138.043894,NaN,138.043894
3,3,2002,22,182.570443,NaN,NaN,142.726956,176.253422,142.726956
4,4,2003,23,184.200432,NaN,NaN,147.303585,182.639416,147.303585


In [8]:
df_1980_group_1 = df_1980[['year', 'group_1']]
df_1980_group_1.columns = ['year', 'wage']
df_1980_group_2 = df_1980[['year', 'group_2']]
df_1980_group_2.columns = ['year', 'wage']
df_1980_group_3 = df_1980[['year', 'group_3']]
df_1980_group_3.columns = ['year', 'wage']
df_1980_group_4 = df_1980[['year', 'group_4']]
df_1980_group_4.columns = ['year', 'wage']
df_1980_group_5 = df_1980[['year', 'group_5']]
df_1980_group_5.columns = ['year', 'wage']
df_1980_group_6 = df_1980[['year', 'group_6']]
df_1980_group_6.columns = ['year', 'wage']

In [12]:
pension_1980_group_1 = Pension(df_1980_group_1, rp_entitlement_year=2035)
pension_1980_group_2 = Pension(df_1980_group_2, military_year=2001, rp_entitlement_year=2036)
pension_1980_group_3 = Pension(df_1980_group_3, rp_entitlement_year=2035)
pension_1980_group_4 = Pension(df_1980_group_4, delivery_year=2009, rp_entitlement_year=2045)
pension_1980_group_5 = Pension(df_1980_group_5, military_year=2000, rp_entitlement_year=2041)
pension_1980_group_6 = Pension(df_1980_group_6, delivery_year=2009, rp_entitlement_year=2041)

In [13]:
pension_1980_group_1.national_pension()

수급시점: 2045
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2045)의 노령연금액: 1340.5586260186985
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 110.69287580705402


/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['wage_with_upper_limit'] = self.wage_history['wage']\
/tmp/ipykernel_13182/4094718912.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['standard_wage'] = self.wage_history['wage_with_upper_limit']\
/tmp/ipykernel_13182/4094718912.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

(264.44848484848484,
     국민연금 월지급액    소득대체율    평균임금대비
 0  110.692876  0.41858  0.434664)

In [14]:
pension_1980_group_1.dc()

퇴직연금 적립액: 9914.492723176274
퇴직연금 월지급액: 40.9330741079124
수급시점: 2045
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2045)의 노령연금액: 1340.5586260186985
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 110.69287580705402


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\


,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,110.692876,0.41858,0.434664,40.933074,151.62595,264.448485,0.573367,0.595399


In [375]:
pension_1980_group_2.national_pension()

entitlement_year: 2045
A_base: 254.6629
A: 601.8308211038989
B_2020: 410.8633333333334
B: 674.0648475724275
Y: 1.8
C: 0.5
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2045의 노령연금액(annuity): 1618.5279594018934
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 133.6454153686669


,연금액,소득대체율,평균임금대비
0,133.645415,0.325279,0.420269


In [376]:
pension_1980_group_3.national_pension()

entitlement_year: 2045
A_base: 254.6629
A: 601.8308211038989
B_2020: 226.05806451612904
B: 370.8722157422561
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2045의 노령연금액(annuity): 1283.8153312340658
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 106.00745708640287


,연금액,소득대체율,평균임금대비
0,106.007457,0.468939,0.333357


In [377]:
pension_1980_group_4.national_pension()

entitlement_year: 2045
A_base: 254.6629
A: 601.8308211038989
B_2020: 172.9838709677419
B: 283.7983756553908
Y: 1.485
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2045의 노령연금액(annuity): 1237.3949367676057
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 102.17442296177768


,연금액,소득대체율,평균임금대비
0,102.174423,0.590659,0.321303


In [378]:
pension_1980_group_5.national_pension()

entitlement_year: 2045
A_base: 254.6629
A: 601.8308211038989
B_2020: 235.6516129032258
B: 386.61144873431454
Y: 1.8
C: 0.5
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2045의 노령연금액(annuity): 1344.1146110668403
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 110.9865013178418


,연금액,소득대체율,평균임금대비
0,110.986501,0.470977,0.349014


In [379]:
pension_1980_group_6.national_pension()

entitlement_year: 2045
A_base: 254.6629
A: 601.8308211038989
B_2020: 164.74285714285716
B: 270.2781189738182
Y: 1.485
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2045의 노령연금액(annuity): 900.6417732808465
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 74.36797318779185


,연금액,소득대체율,평균임금대비
0,74.367973,0.451418,0.233862


In [2]:
df_1970 = pd.read_csv('../data/new/wage_history_1970.csv')
df_1970.head()

,Unnamed: 0,year,age,group_1,group_2,group_3,group_4,group_5,group_6
0,0,1989,19,NaN,NaN,NaN,128.784204,NaN,128.784204
1,1,1990,20,NaN,NaN,NaN,133.467266,NaN,133.467266
2,2,1991,21,NaN,NaN,NaN,138.043894,NaN,138.043894
3,3,1992,22,182.570443,NaN,NaN,142.726956,176.253422,142.726956
4,4,1993,23,184.200432,NaN,NaN,147.303585,182.639416,147.303585


In [3]:
df_1970_group_1 = df_1970[['year', 'group_1']]
df_1970_group_1.columns = ['year', 'wage']
df_1970_group_2 = df_1970[['year', 'group_2']]
df_1970_group_2.columns = ['year', 'wage']
df_1970_group_3 = df_1970[['year', 'group_3']]
df_1970_group_3.columns = ['year', 'wage']
df_1970_group_4 = df_1970[['year', 'group_4']]
df_1970_group_4.columns = ['year', 'wage']
df_1970_group_5 = df_1970[['year', 'group_5']]
df_1970_group_5.columns = ['year', 'wage']
df_1970_group_6 = df_1970[['year', 'group_6']]
df_1970_group_6.columns = ['year', 'wage']

In [4]:
pension_1970_group_1 = Pension(df_1970_group_1, rp_entitlement_year=2025)
pension_1970_group_2 = Pension(df_1970_group_2, military_year=1991, rp_entitlement_year=2026)
pension_1970_group_3 = Pension(df_1970_group_3, rp_entitlement_year=2025)
pension_1970_group_4 = Pension(df_1970_group_4, delivery_year=1999, rp_entitlement_year=2035)
pension_1970_group_5 = Pension(df_1970_group_5, military_year=1990, rp_entitlement_year=2031)
pension_1970_group_6 = Pension(df_1970_group_6, delivery_year=1999, rp_entitlement_year=2031)

In [5]:
pension_1970_group_1.national_pension()[1]

수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2035)의 노령연금액: 1547.4352537758602
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 127.77513421727758


/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['wage_with_upper_limit'] = self.wage_history['wage']\
/tmp/ipykernel_13182/4094718912.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['standard_wage'] = self.wage_history['wage_with_upper_limit']\
/tmp/ipykernel_13182/4094718912.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,국민연금 월지급액,소득대체율,평균임금대비
0,127.775134,0.483176,0.501742


In [6]:
pension_1970_group_1.dc()

퇴직연금 적립액: 2960.14249451475
퇴직연금 월지급액: 12.221273995663877
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2035)의 노령연금액: 1547.4352537758602
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 127.77513421727758


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\


,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,127.775134,0.483176,0.501742,12.221274,139.996408,264.448485,0.52939,0.549732


In [48]:
import docx
doc = docx.Document('./results.docx')

In [58]:
results_1970 = pd.concat([pension_1970_group_1.dc(), pension_1970_group_2.dc(), pension_1970_group_3.dc(),
                          pension_1970_group_4.dc(), pension_1970_group_5.dc(), pension_1970_group_6.dc()])
results_1970.index = [1, 2, 3, 4, 5, 6]
results_1970.index.name = '집단'
results_1970.to_csv('../data/new/results_1970.csv')

퇴직연금 적립액: 2960.14249451475
퇴직연금 월지급액: 12.221273995663877
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2035)의 노령연금액: 1547.4352537758602
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 127.77513421727758
퇴직연금 적립액: 5684.580943002631
퇴직연금 월지급액: 23.469417902584123
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 410.8633333333334
Y: 2.4
C: 0.5
X: 1.2
P: 30
n: 10
2020년 기준 수급시점(2035)의 노령연금액: 1731.8568142208935
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 143.00322830452598
퇴직연금 적립액: 2599.780318300097
퇴직연금 월지급액: 10.733479100197219
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 226.05806451612904
Y: 0
C: 0
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2035)의 노령연금액: 1345.3115356816638
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 111.08533401726096
퇴직연금 적립액: 3455.0643193664127
퇴직연금 월지급액: 14.264613206243897
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 172.9838709677419
Y: 1.8
C: 1
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2

In [59]:
t = doc.add_table(results_1970.shape[0]+1, results_1970.shape[1])

# add the header rows.
for j in range(results_1970.shape[-1]):
    t.cell(0,j).text = results_1970.columns[j]

# add the rest of the data frame
for i in range(results_1970.shape[0]):
    for j in range(results_1970.shape[-1]):
        t.cell(i+1,j).text = str(results_1970.values[i,j])

# save the doc
doc.save('./results1970.docx')

In [43]:
results_1980 = pd.concat([pension_1980_group_1.dc(), pension_1980_group_2.dc(), pension_1980_group_3.dc(),
                          pension_1980_group_4.dc(), pension_1980_group_5.dc(), pension_1980_group_6.dc()])
results_1980.index = [1, 2, 3, 4, 5, 6]
results_1980.index.name = '집단'
results_1980.to_csv('../data/new/results_1980.csv')

퇴직연금 적립액: 9914.492723176274
퇴직연금 월지급액: 40.9330741079124
수급시점: 2045
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2045)의 노령연금액: 1340.5586260186985
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 110.69287580705402
퇴직연금 적립액: 16156.367323229659
퇴직연금 월지급액: 66.70334019313758
수급시점: 2045
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 410.8633333333334
Y: 1.8
C: 0.5
X: 1.2
P: 30
n: 10
2020년 기준 수급시점(2045)의 노령연금액: 1503.136826498594
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 124.11731559310213
퇴직연금 적립액: 8376.399571643751
퇴직연금 월지급액: 34.58288729407979
수급시점: 2045
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 226.05806451612904
Y: 0
C: 0
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2045)의 노령연금액: 1165.0590516709256
2020년 가치로 환산한 수급시점 2045의 노령연금 월지급액(annuity): 96.2014897457346
퇴직연금 적립액: 9111.577164426428
퇴직연금 월지급액: 37.61814887811464
수급시점: 2045
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 172.9838709677419
Y: 1.485
C: 1
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2045)의

In [44]:
results_1990 = pd.concat([pension_1990_group_1.dc(), pension_1990_group_2.dc(), pension_1990_group_3.dc(),
                          pension_1990_group_4.dc(), pension_1990_group_5.dc(), pension_1990_group_6.dc()])
results_1990.index = [1, 2, 3, 4, 5, 6]

results_1990.index.name = '집단'
results_1990.to_csv('../data/new/results_1990.csv')

퇴직연금 적립액: 17778.479575734866
퇴직연금 월지급액: 73.40040911003084
수급시점: 2055
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2055)의 노령연금액: 1218.3078927743993
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 100.59836373597676
퇴직연금 적립액: 28787.418499940926
퇴직연금 월지급액: 118.8520247817645
수급시점: 2055
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 410.8633333333334
Y: 1.455
C: 0.5
X: 1.2
P: 30
n: 10
2020년 기준 수급시점(2055)의 노령연금액: 1392.9252187925968
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 115.01690061123206
퇴직연금 적립액: 14845.030870587796
퇴직연금 월지급액: 61.289343360912355
수급시점: 2055
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 226.05806451612904
Y: 0
C: 0
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2055)의 노령연금액: 1069.5157581905185
2020년 가치로 환산한 수급시점 2055의 노령연금 월지급액(annuity): 88.3122697488196
퇴직연금 적립액: 14216.787054796501
퇴직연금 월지급액: 58.69556964120315
수급시점: 2055
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 172.9838709677419
Y: 1.335
C: 1
X: 1.2
P: 31
n: 11
2020년 기준 수급시

/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [45]:
results_2000 = pd.concat([pension_2000_group_1.dc(), pension_2000_group_2.dc(), pension_2000_group_3.dc(),
                          pension_2000_group_4.dc(), pension_2000_group_5.dc(), pension_2000_group_6.dc()])
results_2000.index = [1, 2, 3, 4, 5, 6]
results_2000.index.name = '집단'
results_2000.to_csv('../data/new/results_2000.csv')

퇴직연금 적립액: 25142.20621295351
퇴직연금 월지급액: 103.80236476905063
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 264.44848484848484
Y: 0
C: 0
X: 1.2
P: 33
n: 13
2020년 기준 수급시점(2065)의 노령연금액: 1173.534535057538
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 96.90132906026375
퇴직연금 적립액: 34464.2164428651
퇴직연금 월지급액: 142.28930971215445
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 410.8633333333334
Y: 1.305
C: 0.5
X: 1.2
P: 30
n: 10
2020년 기준 수급시점(2065)의 노령연금액: 1353.579395845747
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 111.76802942540657
퇴직연금 적립액: 19358.290723224374
퇴직연금 월지급액: 79.92283326043933
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 226.05806451612904
Y: 0
C: 0
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2065)의 노령연금액: 1035.2643888296177
2020년 가치로 환산한 수급시점 2065의 노령연금 월지급액(annuity): 85.4840588065293
퇴직연금 적립액: 21150.93503508549
퇴직연금 월지급액: 87.32396254817465
수급시점: 2065
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 172.9838709677419
Y: 1.2
C: 1
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2065)의

In [ ]:
pension_1970_group_1.dc()

In [26]:
pension_1970_group_2.national_pension()

수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 410.8633333333334
Y: 2.4
C: 0.5
X: 1.2
P: 30
n: 10
2020년 기준 수급시점(2035)의 노령연금액: 1731.8568142208935
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 143.00322830452598


/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['wage_with_upper_limit'] = self.wage_history['wage']\
/tmp/ipykernel_13182/4094718912.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['standard_wage'] = self.wage_history['wage_with_upper_limit']\
/tmp/ipykernel_13182/4094718912.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

(410.8633333333334,
     국민연금 월지급액     소득대체율    평균임금대비
 0  143.003228  0.348055  0.561539)

In [27]:
pension_1970_group_2.dc()

퇴직연금 적립액: 5684.580943002631
퇴직연금 월지급액: 23.469417902584123
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 410.8633333333334
Y: 2.4
C: 0.5
X: 1.2
P: 30
n: 10
2020년 기준 수급시점(2035)의 노령연금액: 1731.8568142208935
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 143.00322830452598


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\


,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,143.003228,0.348055,0.561539,23.469418,166.472646,410.863333,0.405178,0.653698


In [28]:
pension_1970_group_3.dc()

퇴직연금 적립액: 2599.780318300097
퇴직연금 월지급액: 10.733479100197219
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 226.05806451612904
Y: 0
C: 0
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2035)의 노령연금액: 1345.3115356816638
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 111.08533401726096


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,111.085334,0.491402,0.436205,10.733479,121.818813,226.058065,0.538883,0.478353


In [29]:
pension_1970_group_4.dc()

퇴직연금 적립액: 3455.0643193664127
퇴직연금 월지급액: 14.264613206243897
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 172.9838709677419
Y: 1.8
C: 1
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2035)의 노령연금액: 1245.2276124359398
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 102.82118422843469


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,102.821184,0.594398,0.403754,14.264613,117.085797,172.983871,0.67686,0.459768


In [30]:
pension_1970_group_5.dc()

퇴직연금 적립액: 4883.398276651369
퇴직연금 월지급액: 20.161647109725575
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 235.6516129032258
Y: 2.4
C: 0.5
X: 1.2
P: 31
n: 11
2020년 기준 수급시점(2035)의 노령연금액: 1362.6412018339913
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 112.51628268742418


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,112.516283,0.477469,0.441824,20.161647,132.67793,235.651613,0.563026,0.520994


In [31]:
pension_1970_group_6.dc()

퇴직연금 적립액: 301.92845773140795
퇴직연금 월지급액: 1.2465448591955848
수급시점: 2035
2020년 기준 A값: 254.66290897019599
2020년 기준 B값: 164.74285714285716
Y: 1.8
C: 1
X: 1.2
P: 21
n: 1
2020년 기준 수급시점(2035)의 노령연금액: 961.5696691856382
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 79.39892363164073


/tmp/ipykernel_13182/4094718912.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_saving'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.wage_history['annual_reserve'] = self.wage_history\
/tmp/ipykernel_13182/4094718912.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,국민연금 월지급액,소득대체율,평균임금대비,퇴직연금 월지급액,합산연금액,생애평균임금,합산소득대체율,평균임금대비
0,79.398924,0.481957,0.31178,1.246545,80.645468,164.742857,0.489523,0.316675


In [387]:
pension_1970_group_4.national_pension()

entitlement_year: 2035
A_base: 254.6629
A: 426.6491917509541
B_2020: 172.9838709677419
B: 232.81351497623046
Y: 1.8
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2035의 노령연금액(annuity): 1293.295496528109
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 106.79025519694908


,연금액,소득대체율,평균임금대비
0,106.790255,0.617342,0.335818


In [388]:
pension_1970_group_5.national_pension()

entitlement_year: 2035
A_base: 254.6629
A: 426.6491917509541
B_2020: 235.6516129032258
B: 317.15604468146563
Y: 2.4
C: 0.5
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2035의 노령연금액(annuity): 1405.05139287892
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 116.01818549060994


,연금액,소득대체율,평균임금대비
0,116.018185,0.492329,0.364837


In [389]:
pension_1970_group_6.national_pension()

entitlement_year: 2035
A_base: 254.6629
A: 426.6491917509541
B_2020: 164.74285714285716
B: 221.7221953936267
Y: 1.8
C: 1
X: 1.2
P: 33
n: 13
2020년 가치로 환산한 수급시점 2035의 노령연금액(annuity): 970.3610065035779
2020년 가치로 환산한 수급시점 2035의 노령연금 월지급액(annuity): 80.12484370035322


,연금액,소득대체율,평균임금대비
0,80.124844,0.486363,0.251965


In [391]:
df_1970_group_1.head()

,year,wage,wage_with_upper_limit,standard_wage,P_n,A+B,(A+B)*P_n
0,1989,NaN,NaN,NaN,2.4,693.583824,NaN
1,1990,NaN,NaN,NaN,2.4,693.583824,NaN
2,1991,NaN,NaN,NaN,2.4,693.583824,NaN
3,1992,182.570443,182.570443,182.5,2.4,693.583824,1664.601177
4,1993,184.200432,184.200432,184.2,2.4,693.583824,1664.601177


In [392]:
1/(1-1/1.026)

39.461538461538375

In [394]:
1/0.95

1.0526315789473684